In [8]:
import mysql.connector
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

# Database connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="HunnyS@1511",
    database="patient_readmission"
)

# Create a cursor object to execute SQL queries
cursor = db.cursor()

# SQL query to retrieve patient features data
query = """
    WITH patient_medications AS (
        SELECT 
            patient_id,
            medication_name,
            start_date,
            end_date,
            DATEDIFF(end_date, start_date) AS duration
        FROM 
            medications
    ),
    lab_result_averages AS (
        SELECT 
            patient_id,
            AVG(result_value) AS result
        FROM 
            lab_results
        GROUP BY 
            patient_id
    ),
    comorbidity_index AS (
        SELECT 
            patient_id,
            COUNT(icd_code) AS comorbidity_index
        FROM 
            diagnoses
        GROUP BY 
            patient_id
    )
    SELECT 
        p.patient_id,
        p.age,
        p.gender,
        ci.comorbidity_index,
        pm.duration,
        lra.result,
        r.readmission_risk
    FROM 
        patients p
    JOIN 
        comorbidity_index ci ON p.patient_id = ci.patient_id
    JOIN 
        patient_medications pm ON p.patient_id = pm.patient_id
    JOIN 
        lab_result_averages lra ON p.patient_id = lra.patient_id
    JOIN 
        readmission_risk r ON p.patient_id = r.patient_id;
"""

# Execute the SQL query
cursor.execute(query)

# Fetch all the rows from the query result
data = cursor.fetchall()

# Create pandas dataframe from the retrieved data
df = pd.DataFrame(data, columns=['patient_id', 'age', 'gender', 'comorbidity_count', 'duration', 'result', 'readmission_risk'])

# Convert gender to numerical value
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1})

# Feature engineering
df['age_group'] = pd.cut(df['age'], bins=[0, 18, 35, 50, 65, 100], labels=[0, 1, 2, 3, 4])
df['comorbidity_group'] = pd.cut(df['comorbidity_count'], bins=[0, 1, 3, 5, np.inf], labels=[0, 1, 2, 3])
df['duration_group'] = pd.cut(df['duration'], bins=[-np.inf, 7, 14, 30, np.inf], labels=[0, 1, 2, 3])

# Additional feature engineering
df['age_comorbidity_interaction'] = df['age'] * df['comorbidity_count']
df['duration_result_ratio'] = df['duration'] / (df['result'] + 1)  # Adding 1 to avoid division by zero

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=['age_group', 'comorbidity_group', 'duration_group'])

# Split data into features and target
X = df.drop(['patient_id', 'readmission_risk'], axis=1)
y = df['readmission_risk']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'classifier__max_depth': [5, 10, 15, 20, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__criterion': ['gini', 'entropy']
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2)

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Fit the model
grid_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters
print("Best parameters:", grid_search.best_params_)

# Make predictions
y_pred = grid_search.predict(X_test)

# Evaluate the model
print('Decision Tree Model:')
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

# Feature importance
feature_importance = grid_search.best_estimator_.named_steps['classifier'].feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)
print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

# Close the cursor and connection
cursor.close()
db.close()

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters: {'classifier__criterion': 'entropy', 'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}
Decision Tree Model:
Accuracy: 0.508
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.52      0.51      1000
           1       0.51      0.50      0.50      1000

    accuracy                           0.51      2000
   macro avg       0.51      0.51      0.51      2000
weighted avg       0.51      0.51      0.51      2000

Confusion Matrix:
[[517 483]
 [501 499]]

Top 10 Most Important Features:
                        feature  importance
4                        result    0.409878
6         duration_result_ratio    0.279102
0                           age    0.120918
5   age_comorbidity_interaction    0.106891
1                        gender    0.042323
3                      duration    0.016378
9            